# Study of effect of national factors on home prices in US
Task - Using publically available data for the national factors that impact supply and demand of homes in US, build a model to study the effect of these variables on home prices.

Approach - The following variables are chosen for the study-

Unemployment Rate

Per Capita 

Median Household Income

Construction Prices

CPI

Interest Rates

Housing subsidies

Number of Households

As a proxy to the home prices, S&P CASE-SHILLER Index is used.

Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed and combined to create a datset. Data for different variables had different frequencies. So, to combine the data, necessary interpolations are made.

Linear Regression is used as most of the variables have high correlation with the target variable.

In [1]:
import numpy as np
import pandas as pd

In [4]:
#Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv(r"C:\Users\Dell\Downloads\CASE-SCHILLER Home Price Index.csv")

#Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

#Selecting data post JUNE 2001
mask = df_CS["DATE"] >= "2001-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print(df_CS.shape)
df_CS.tail()

(265, 4)


,DATE,CSUSHPISA,Year,Month
260,2023-03-01,298.637,2023,3
261,2023-04-01,300.213,2023,4
262,2023-05-01,302.566,2023,5
263,2023-06-01,304.593,2023,6
264,2023-07-01,306.767,2023,7


In [6]:
df_unemp = pd.read_csv(r"C:\Users\Dell\Downloads\Unemployment Rate.csv")
df_unemp.drop([265,266], inplace = True)
print(df_unemp.shape)
df_unemp.tail()

(437, 2)


,DATE,UNRATE
434,2023-03-01,3.5
435,2023-04-01,3.4
436,2023-05-01,3.7
437,2023-06-01,3.6
438,2023-07-01,3.5


In [7]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv(r"C:\Users\Dell\Downloads\Per Capita GDP.csv", names = ["DATE", "Per_Capita_GDP"], skiprows = 1)
print(df_pcgdp.shape)
df_pcgdp.tail()

(147, 2)


,DATE,Per_Capita_GDP
142,2022-07-01,65462.0
143,2022-10-01,65783.0
144,2023-01-01,66078.0
145,2023-04-01,66341.0
146,2023-07-01,67083.0


In [8]:
# Interest Rate Data
df_Fed_rate = pd.read_csv(r"C:\Users\Dell\Downloads\Interest Rate.csv").drop([265,266])
print(df_Fed_rate.shape)
df_Fed_rate.tail()

(437, 2)


,DATE,FEDFUNDS
434,2023-03-01,4.65
435,2023-04-01,4.83
436,2023-05-01,5.06
437,2023-06-01,5.08
438,2023-07-01,5.12


In [11]:
# Reading Per Capita GDP Data into a dataframe
df_cons_price_index = pd.read_csv(r"C:\Users\Dell\Downloads\Construction price index.csv", names = ["DATE", "Cons_Materials"], skiprows = 1)
df_cons_price_index.drop([265], inplace = True)
print(df_cons_price_index.shape)
df_cons_price_index.tail()

(438, 2)


,DATE,Cons_Materials
434,2023-03-01,331.729
435,2023-04-01,333.366
436,2023-05-01,337.473
437,2023-06-01,337.336
438,2023-07-01,334.576


In [15]:
# Consumer Price Index
df_CPI = pd.read_csv(r"C:\Users\Dell\Downloads\Construction price index.csv", names = ["DATE", "CPI"], skiprows = 1).drop([265])
print(df_CPI.shape)
df_CPI.tail()

(438, 2)


,DATE,CPI
434,2023-03-01,331.729
435,2023-04-01,333.366
436,2023-05-01,337.473
437,2023-06-01,337.336
438,2023-07-01,334.576


In [16]:
# Housing Subsidies

df_subsidy = pd.read_csv(r"C:\Users\Dell\Downloads\Housing Subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
print(df_subsidy.shape)
df_subsidy.tail()

(36, 2)


,DATE,Subsidy
31,2018-01-01,38.859
32,2019-01-01,40.185
33,2020-01-01,44.147
34,2021-01-01,45.299
35,2022-01-01,48.021


In [17]:
# Real Median Household Income

df_income = pd.read_csv(r"C:\Users\Dell\Downloads\Income.csv", names = ["DATE", "Income"], skiprows = 1)
print(df_income.shape)
df_income.tail()

(439, 2)


,DATE,Income
434,2023-03-01,16730.2
435,2023-04-01,16763.9
436,2023-05-01,16818.5
437,2023-06-01,16809.5
438,2023-07-01,16796.9


In [18]:
# Number of households

df_households = pd.read_csv(r"C:\Users\Dell\Downloads\Total Houseolds.csv", names = ["DATE", "Num_Households"], skiprows = 1)
print(df_households.shape)
df_households.tail()

(37, 2)


,DATE,Num_Households
32,2019-01-01,128579.0
33,2020-01-01,128451.0
34,2021-01-01,129224.0
35,2022-01-01,131202.0
36,2023-01-01,131434.0


In [24]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")

In [25]:
df.head()

,DATE,CSUSHPISA,Year,Month,Per_Capita_GDP
0,2001-07-01,113.491,2001,7,49774.0
1,2001-08-01,114.167,2001,8,NaN
2,2001-09-01,114.812,2001,9,NaN
3,2001-10-01,115.310,2001,10,49783.0
4,2001-11-01,115.857,2001,11,NaN


In [29]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_CPI, df_unemp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)

In [32]:
print(df.shape)
df.head()

(439, 8)


,CSUSHPISA,Year,Month,Per_Capita_GDP,CPI,UNRATE,Cons_Materials,FEDFUNDS
DATE,,,,,,,,
1987-01-01,NaN,NaN,NaN,NaN,107.6,6.6,107.6,6.43
1987-02-01,NaN,NaN,NaN,NaN,107.9,6.6,107.9,6.10
1987-03-01,NaN,NaN,NaN,NaN,108.1,6.6,108.1,6.13
1987-04-01,NaN,NaN,NaN,NaN,108.3,6.3,108.3,6.37
1987-05-01,NaN,NaN,NaN,NaN,108.3,6.3,108.3,6.85


In [35]:
# Merging other dataframes 
others = [ df_households, df_income, df_subsidy]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,CPI,UNRATE,Cons_Materials,FEDFUNDS,Num_Households,Income,Subsidy
DATE,,,,,,,,,,,
2001-07-01,NaN,NaN,NaN,NaN,107.6,6.6,107.6,6.43,NaN,NaN,NaN
2001-08-01,NaN,NaN,NaN,NaN,107.9,6.6,107.9,6.10,NaN,NaN,NaN
2001-09-01,NaN,NaN,NaN,NaN,108.1,6.6,108.1,6.13,NaN,NaN,NaN
2001-10-01,NaN,NaN,NaN,NaN,108.3,6.3,108.3,6.37,NaN,NaN,NaN
2001-11-01,NaN,NaN,NaN,NaN,108.3,6.3,108.3,6.85,NaN,NaN,NaN


In [36]:
df.isna().sum()

CSUSHPISA          174
Year               174
Month              174
Per_Capita_GDP    2266
CPI                 12
UNRATE              24
Cons_Materials      12
FEDFUNDS            24
Num_Households     174
Income             174
Subsidy            223
dtype: int64

In [37]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [38]:
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,CPI,UNRATE,Cons_Materials,FEDFUNDS,Num_Households,Income,Subsidy
DATE,,,,,,,,,,,
2001-07-01,NaN,NaN,NaN,NaN,107.6,6.6,107.6,6.43,NaN,NaN,NaN
2001-08-01,NaN,NaN,NaN,NaN,107.9,6.6,107.9,6.10,NaN,NaN,NaN
2001-09-01,NaN,NaN,NaN,NaN,108.1,6.6,108.1,6.13,NaN,NaN,NaN
2001-10-01,NaN,NaN,NaN,NaN,108.3,6.3,108.3,6.37,NaN,NaN,NaN
2001-11-01,NaN,NaN,NaN,NaN,108.3,6.3,108.3,6.85,NaN,NaN,NaN


In [39]:
df.dropna(inplace = True)

In [40]:
df.isna().sum()

CSUSHPISA         0
Year              0
Month             0
Per_Capita_GDP    0
CPI               0
UNRATE            0
Cons_Materials    0
FEDFUNDS          0
Num_Households    0
Income            0
Subsidy           0
dtype: int64

In [41]:
df.shape

(3072, 11)

In [42]:
df.tail()

,CSUSHPISA,Year,Month,Per_Capita_GDP,CPI,UNRATE,Cons_Materials,FEDFUNDS,Num_Households,Income,Subsidy
DATE,,,,,,,,,,,
NaT,297.413,2022.0,12.0,66019.0,326.449,3.5,326.449,4.1,131202.0,16161.4,48.021
NaT,297.413,2022.0,12.0,66030.8,326.449,3.5,326.449,4.1,131202.0,16184.9,48.021
NaT,297.413,2022.0,12.0,66042.6,326.449,3.5,326.449,4.1,131202.0,16223.5,48.021
NaT,297.413,2022.0,12.0,66054.4,326.449,3.5,326.449,4.1,131202.0,16229.6,48.021
NaT,297.413,2022.0,12.0,66066.2,326.449,3.5,326.449,4.1,131202.0,16265.1,48.021


In [43]:
df.to_csv("prepared_dataset.csv")

In [44]:
df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,CPI,UNRATE,Cons_Materials,FEDFUNDS,Num_Households,Income,Subsidy
DATE,,,,,,,,,,,
2016-01-01,113.491,2001.0,7.0,49774.0,143.4,4.6,143.4,3.77,108209.0,10211.7,20.573
2016-02-01,113.491,2001.0,7.0,49774.0,143.4,4.6,143.4,3.77,108209.0,10223.4,20.573
2016-03-01,113.491,2001.0,7.0,49774.0,143.4,4.6,143.4,3.77,108209.0,10254.4,20.573
2016-04-01,113.491,2001.0,7.0,49774.0,143.4,4.6,143.4,3.77,108209.0,10233.1,20.573
2016-05-01,113.491,2001.0,7.0,49774.0,143.4,4.6,143.4,3.77,108209.0,10214.7,20.573
